<a href="https://colab.research.google.com/github/fbeilstein/dbms/blob/master/DB_lecture_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Concurrency**.  Consider multiplier and adder to complete 
```
i += 2
i *= 2
```
Reason about order of read-write to *i* and possible outcomes.

To define **execution histories** precisely and reduce the number of possible outcomes, we need **consistency models**. Consistency models describe **concurrent executions** and establish an order in which operations can be executed and made visible to the participants. Using different consistency models, we can constraint or relax the number of states the system can be in.

We often use the terms **concurrent** and **parallel** computing interchangeably, but these concepts have a slight semantic difference. 

When two sequences of steps execute **concurrently**, both of them are in progress, but only one of them is executed at any moment. In a concurrent system, we can have **shared memory**, which processors can use to exchange the information.

If two sequences execute in **parallel**, their steps can be executed simultaneously. Concurrent operations overlap in time, while parallel operations are executed by multiple processors. **No shared memory**.


Joe Armstrong, creator of the Erlang programming language, gave an example: concurrent execution is like having two queues to a single coffee machine, while parallel execution is like having two queues to two coffee machines.

We can try to introduce some notion of *shared memory* to a distributed system, for example, a single source of information, such as database. Even if we solve the problems with concurrent access to it, *we still cannot guarantee that all processes are in sync*.

What if no answer from the server for a *long time*? The system has to be described in terms of **synchrony**: whether the communication is **fully asynchronous**, or whether there are some **timing assumptions**. These timing assumptions allow us to introduce **operation timeouts and retries**.

Processes may crash by failing to participate in further algorithm steps, having a temporary failure, or by omitting some of the messages. We need to define a **failure model** and describe ways in which failures can occur before we decide how to treat them.

A property that describes system reliability and whether or not it can continue operating correctly in the presence of failures is called **fault tolerance**. 

Failures are inevitable, so we need to build systems with reliable components, and eliminating a single point of failure in the form of the aforementioned single-node database can be the first step in this direction. We can do this by introducing some **redundancy** and adding a **backup database**. But how do we keep multiple copies of shared state in sync now?

**Fallacies of distributed computing**

* network is reliable
* a successful initial connection to the server guarantees that the link is stable
* latency is zero
* instantaneous execution
* bandwidth is infinite
* queue capacity is not infinite
* we always get a positive response from server

In general, **process-local queues** are used to achieve the following goals:
* **Decoupling**
Receipt and processing are separated in time and happen independently. 
* **Pipelining**
Requests in different stages are processed by independent parts of the system. The subsystem responsible for receiving messages doesn’t have to block until the previous message is fully processed.
* **Absorbing short-time bursts**
System load tends to vary, but request inter-arrival times are hidden from the component responsible for request processing. Overall system latency increases because of the time spent in the queue, but this is usually still better than responding with a failure and retrying the request

To handle queue overflow we often use (highly unpopular but often used) **backpressure**.

Strategies:
* Control the producer (slow down/speed up is decided by consumer)
* Buffer (accumulate incoming data spikes temporarily)
* Drop (sample a percentage of the incoming data)

**Clocks and Time**

When collecting and aggregating data from participants with a different perception of time, you should understand time drifts between them and normalize times accordingly, rather than relying on the source timestamp. Unless you use specialized highprecision time sources, you should not rely on timestamps for synchronization or ordering. Of course this doesn’t mean we cannot or should not rely on time at all: in the end, any synchronous system uses local clocks for timeouts.

But we cannot fully remove clocks from consideration. For example, [Spanner](https://en.wikipedia.org/wiki/Spanner_(database)) uses a special time API that returns a timestamp and uncertainty bounds to impose a strict transaction order. Some failure-detection algorithms rely on a shared notion of time and a guarantee that the clock drift is always within allowed bounds for correctness

**State Consistency**

Distributed algorithms do not always guarantee **strict state consistency**. Some approaches have looser constraints and allow state divergence between replicas, and rely on **conflict resolution** (an ability to detect and resolve diverged states within the system) and **read-time data repair** (bringing replicas back in sync during reads in cases where they respond with different results). 

Assuming that the state is fully consistent across the nodes may lead to subtle bugs:
* there was a bug in Apache Cassandra, caused by the fact that schema changes (note: NoSQL = not only SQL, but not without schemes at all) propagate to servers at different times. If you tried to read from the database while the schema was propagating, there was a chance of corruption, since one server
encoded results assuming one schema and the other one decoded them using a different schema.
* a bug caused by the divergent view of the ring: if one of the nodes assumes that the other node holds data records for a key, but this other node has a different view of the cluster, reading or writing the data can result in misplacing data records or getting an empty response while data records are in fact happily present on the other node.

**Hiding complexity behind an API might be dangerous**

Simply hiding local and remote behind the same interface, however useful, might be misleading. Additional API parameters may be necessary for debugging, configuration, and observability. 

Problems examples:
* latency: remote invocation is many times more costly than the local one, since it involves two-way network transport, serialization/deserialization, and many other steps. 
* interleaving local and remote calls may lead to performance degradation and unintended side effects.

**Need to Handle Failures**

maintenance, software problems, out-of-memory killer, runtime bugs, hardware issues, etc.

If the remote server doesn’t respond, we do not always know the exact reason for it. It could be caused by the crash, a network failure, the remote process, or the link to it being slow. Some distributed algorithms use **heartbeat protocols** and **failure detectors** to form a hypothesis about which participants are alive and reachable.

When two or more servers cannot communicate with each other, we call the situation **network partition** (need this for CAP theorem, explain it later): independent groups can proceed with execution and produce conflicting results. Network links can also fail **asymmetrically**: messages can still be getting delivered from one process to the other one, but not vice versa.

To build a system that is robust in the presence of failure of one or multiple processes, we have to consider cases of **partial failures** and how the system can continue operating even though a part of it is unavailable or functioning incorrectly.

Opensource to recrate failure scenarios: Toxiproxy (bandwidth, latency, timeouts), Chaos Monkey (randomly services shutdown), CharybdeFS (filesystem), CrashMonkey (filesystem agnostic record-replay-and test framework)

**Cascading Failures**

A process tipping over under a high load increases the load for the rest of cluster, making it even more probable for the other
nodes to fail. **Cascading failures** can propagate from one part of the system to the other, increasing the scope of the problem.

Sometimes, cascading failures can even be initiated by perfectly good intentions. For example, a node was offline for a while and did not receive the most recent updates. Streaming the data it’s missing over to it, may exhaust network resources or cause the node to fail shortly after the startup.

To protect a system from propagating failures and treat failure scenarios gracefully, **circuit breakers** can be used.They  monitor failures and allow fallback mechanisms that can protect the system by steering away from the failing service, giving it some time to recover, and handling failing calls gracefully.

server does not respond -> client starts a reconnection loop -> server overloaded with new connection requests and client-side retries. To avoid that, we can use a **backoff strategy**: clients wait for some time before reconnection. Backoff can help us to avoid amplifying problems by scheduling retries and increasing the time window between subsequent requests. To prevent different clients from retrying all at once after the **backoff period**, we can introduce **jitter**. Jitter adds small random time periods to backoff and reduces the probability of clients waking up and retrying at the same time.

We should equip our systems with circuit breakers, backoff, validation, and coordination mechanisms.

**Links**

Two processes, connected with a **link**. Processes can send messages to each other. Any communication medium is imperfect, and messages can get lost or delayed.

**Fair-loss link**

After the message M is sent, from the senders’ perspective, it can be in one of the following states:
* Not yet delivered to process B (but will be, at some point in time)
* Irrecoverably lost during transport
* Successfully delivered to the remote process

the sender **does not have any way** to find out if the message is already delivered. In distributed systems terminology, this kind of link is called **fair-loss**.

Definition, fair-link properties:
* **Fair loss**
If both sender and recipient are correct and the sender keeps retransmitting the message infinitely many times, it will eventually be delivered.
* **Finite duplication**
Sent messages won’t be delivered infinitely many times.
* **No creation**
A link will not come up with messages; in other words, it won’t deliver the message that was never sent.

Think about UDP protocol as analogy.

**Message acknowledgments**

**Message acknowledgments**: a way for the recipient to notify the sender that it has received the message. We need to use bidirectional communication channels and add some means that allow us to distinguish differences between the messages (e.g. enumerate them).

Note that acknowledgments may be lost as well!

**Message retransmits**

A sent message may still get lost, or the remote process may fail before acknowledging it. To solve this problem and provide delivery guarantees, we can try **retransmits** instead. Retransmits are a way for the sender to retry a **potentially failed** operation. We say potentially failed, because the sender doesn’t really know whether it has failed or not, since the type of link we’re about to discuss does not use
acknowledgments.

**Stubborn link**
send message -> wait time T -> send again -> (cycle)

**Stubborn + retransmits** 
send until acknowledgement

**Message duplicates problem**


Processing duplicates is only safe if the operation we’re about to perform is **idempotent**. An idempotent operation is one that can be executed multiple times, yielding the same result without producing additional side effects.

If every operation was idempotent, we could think less about delivery semantics, however, operations are not necessarily idempotent, and merely assuming that they are might lead to cluster-wide side effects.

Idempotence is particularly important in the presence of partial failures and network partitions, since we cannot always find out the exact status of a remote operation— whether it has succeeded, failed, or will be executed shortly—and we just have to wait longer. 

Equivalent to idempotence is **deduplication** to avoid processing messages more than once.

**Message order**

2 problems: messages can arrive out of order and may arrive more than once.
Suppose messages are enumerated. Track
* $n_{consecutive}$ highest sequence number, up to which it has seen all messages. Messages up to this number can be put back in order.
* $n_{processed}$ the highest sequence number, up to which messages were processed. 

If the received message has a nonconsecutive sequence number, the receiver puts it into the reordering buffer. 

The recipient can safely discard the messages with sequence numbers up to $n_{consecutive}$ (**deduplication**).



**perfect link**

provides the following guarantees:
* **Reliable delivery** 
Every message sent once by the correct process A to the correct process B, will eventually be delivered.
* **No duplication**
No message is delivered more than once.
* **No creation**
Same as with other types of links, it can only deliver the messages that were actually sent.


This might remind you of the TCP protocol (however, reliable delivery in TCP is guaranteed only in the scope of a **single session**). 

TCP has a much more sophisticated model for dealing with acknowledgments, which groups acknowledgments and reduces the protocol-level overhead. In addition, TCP has selective acknowledgments, flow control, congestion control, error detection, and many other features that are out of the scope of our discussion.

**Exactly-once delivery**


There are only two hard problems in distributed systems: 

$2$. Exactly-once delivery 

$1$. Guaranteed order of messages 

$2$. Exactly-once delivery.

— Mathias Verraes

Whether or not **exactly-once delivery** is possible? Most of the
real-world systems employ **at-least-once delivery**, which ensures that the sender retries until it receives an acknowledgment, otherwise the message is not considered
to be received. Another delivery semantic is **at-most-once**, not sure if anyone uses except UPD.


TCP: does what we want, but guarantee is valid only for a single session: if the message is acknowledged and processed, but the sender didn’t receive the acknowledgment before the connection was interrupted, the application is not aware of this delivery and, depending on its logic, it might attempt to send the message once again.


**Exactly-once processing** is what’s interesting here since duplicate deliveries (or packet transmissions) have no side effects and are merely an artifact of the best effort by the link. 

Any misunderstanding about whether or not exactly-once delivery is possible most likely comes from approaching the problem from different protocol and abstraction levels and the definition of “delivery.” It’s not possible to build a reliable link without
ever transferring any message more than once, but we can create the illusion of exactly-once delivery from the sender’s perspective by processing the message once and ignoring duplicates.

**Two Generals’ Problem**

This thought experiment shows that it is impossible to achieve an agreement between two parties if communication is **asynchronous** in the presence of **link failures**. Even
though TCP exhibits properties of a perfect link, it’s important to remember that perfect links, despite the name, do not guarantee perfect delivery. They also can’t guarantee that participants will be alive the whole time, and are concerned only with transport.

Imagine two armies, led by two generals, preparing to attack a fortified city. The armies are located on two sides of the city and can succeed in their siege only if they attack simultaneously. The generals can communicate by sending messengers, and already have devised an attack plan. The only thing they now have to agree on is whether or not to carry out
the plan. They can send messangers, but there is no warranty that any messanger gets to the destination.

**The FLP (Fisher, Lynch, and Paterson) Impossibility Problem**

* **Agreement**
The decision the protocol arrives at has to be unanimous: each process decides on some value, and this has to be the same for all processes. Otherwise, we have not reached a consensus.
* **Validity**
The agreed value has to be proposed by one of the participants, which means that the system should not just “come up” with the value. This also implies nontriviality of the value: processes should not always decide on some predefined default value.
* **Termination**
An agreement is final only if there are no processes that did not reach the decision state.

The paper shows that, given these assumptions, there exists **no** protocol that can guarantee consensus in a bounded time. 


If we do not consider an upper time bound for the process to complete the algorithm steps, process failures can’t be reliably detected, and there’s no deterministic algorithm to reach a consensus

However, FLP Impossibility does not mean we have to pack our things and go home, as reaching consensus is not possible. It only means that we cannot always reach consensus in an asynchronous system in bounded time. In practice, systems exhibit at least some degree of synchrony, and the solution to this problem requires a more refined model.

**System Synchrony**

These assumptions can be loosened up, and the system can be considered to be synchronous. 

A synchronous system can also be represented in terms of synchronized process-local clocks: there is an upper time bound in time difference between the two process-local time sources. It assumes that processes are progressing at comparable rates, that transmission delays are bounded, and message delivery cannot take arbitrarily long.

Properties of both asynchronous and synchronous models can be combined, and we can think of a system as partially synchronous. A partially synchronous system exhibits some of the properties of the synchronous system, but the bounds of message delivery, clock drift, and relative processing speeds might not be exact and hold only most of the time

Designing systems under a synchronous model allows us to use timeouts. We can build more complex abstractions, such as leader election, consensus, failure detection, and many others on top of them. 

**Failure Models**

A **failure model** describes exactly how processes can crash in a distributed system, and algorithms are developed using these assumptions.

* **Crash Faults** 


Most of the time, we assume once the process has crashed, it remains in this state. This model does not assume that it is impossible for the process to recover, and does not discourage recovery or try to prevent it. Nothing prevents processes from recovering, catching up with the system state, and participating in the next instance of the algorithm, BUT not current roud. Assigning the recovering process a new, different identity does not make the model equivalent to crash-recovery, since
most algorithms use predefined lists of processes and clearly define failure semantics in terms of how many failures they can tolerate.

**Crash-recovery** is a different process abstraction, under which the process stops executing the steps required by the algorithm, but recovers at a later point and tries to
execute further steps. The possibility of recovery requires introducing a durable state and recovery protocol into the system. Algorithms that allow crash-recovery need to take all possible recovery states into consideration, since the
recovering process may attempt to continue execution from the last step known to it.

* **Omission Faults**

Another failure model is omission fault. This model assumes that the process skips some of the algorithm steps, or is not able to execute them, or this execution is not visible to other participants, or it cannot send or receive messages to and from other participants. Omission fault **captures network partitions** between the processes caused by faulty network links, switch failures, or network congestion. Network partitions can be represented as omissions of messages between individual processes or process groups. A crash can be simulated by completely omitting any messages to and from the process.

* **Arbitrary** or **Byzantine faults**

a process continues executing the algorithm steps, but in a way that contradicts the algorithm (for example, if a process in a consensus algorithm decides on a value that no other participant has ever proposed).

Reasons: bugs in software, or due to processes running differ‐
ent versions of the algorithm, etc. It can get much more difficult when we do not have control over all processes, and one of the processes is intentionally misleading other processes.

You might have heard of Byzantine fault tolerance from the airspace industry: airplane and spacecraft systems do not take responses from subcomponents at face value and cross-validate their results. Another widespread application is cryptocurrencies, where there is no central authority, different parties control the nodes, and adversary participants have a material incentive to forge values and attempt to game the system by providing faulty responses.

**Consistency models**

Consistency models are important, since they explain visibility semantics and behavior of the system in the presence of multiple copies of data.

**Fault tolerance** is a property of a system that can continue operating correctly in the presence of failures of its components. The primary goal is to remove a single point of failure from the system and make sure that we have redundancy in mission-critical components. Usually, redundancy is entirely **transparent** for the user.

Data **replication** is a way of introducing redundancy by maintaining multiple copies of data in the system. 

quite costly to perform this operation for *every* operation in the database. We can explore some more cost-effective and flexible ways to make data *look* consistent from user's perspective

**Achieving Availability**

intermittent failures should not impact **availability**: from the user’s perspective, the system as a whole has to continue operating as if nothing has happened. In software engineering, we
always strive for high availability, and try to minimize **downtime**.



**Infamous CAP**

**Availability** is a property that measures the ability of the system to serve a response for every request successfully. The theoretical definition of availability mentions eventual response, but of course, in a real-world system, we’d like to avoid services that take indefinitely long to respond.

**Consistency** defined in terms of Linearizability.


**Linearizability** is the strongest single-object, single-operation consistency model. Under this model, effects of the write become visible to all readers exactly once at some point in time between its start and end, and no client can observe state transitions or side effects of partial (i.e., unfinished, still in-flight) or incomplete (i.e., interrupted before completion) write operations.

* If two operations overlap, they may take effect in any order.
* All read operations that occur after write operation completion can observe the effects of this operation. As
soon as a single read operation returns a particular value, all reads that come after it return the value at least as recent as the one it returns.
* This order should be consistent, which means that every read of the shared value should return the latest value written to this shared variable preceding this read, or the value of a write that overlaps with this read.
* Between the two concurrent writes, only one can go first.


Even though operations are concurrent and have some overlap, their effects become visible in a way that makes them appear sequential. No operation happens instantaneously, but still **appears** to be atomic.

**Linearization point**


One of the most important traits of linearizability is visibility: once the operation is complete, everyone must see it, and the system can’t “travel back in time,” reverting it
or making it invisible for some participants. In other words, linearization prohibits stale reads and requires reads to be monotonic.


This consistency model is best explained in terms of atomic (i.e., uninterruptible, indivisible) operations. Operations do not have to be instantaneous (also because there’s no such thing), but their effects have to become visible at some point in time,
making an illusion that they were instantaneous. This moment is called a **linearization point**

**Network partitions**: one part of the system looses connection with other.

**CAP**


**CAP** describes a continuum of potential choices, where on different sides of the spectrum we have systems that are:
* **Consistent and partition tolerant** CP systems prefer failing requests to serving potentially inconsistent data.
* **Available and partition tolerant** AP systems loosen the consistency requirement and allow serving potentially inconsistent values during the request.

**Use CAP Carefully**

CAP discusses **network partitions** rather than **node crashes**
or any other type of failure (such as crash-recovery).

**Harvest and Yield**

CAP conjecture discusses consistency and availability only in their strongest forms: linearizability and the ability of the system to eventually respond to every request.

Instead of being either consistent or available, systems can provide relaxed guarantees.

* **Harvest**
Defines how complete the query is: if the query has to return 100 rows, but can fetch only 99 due to unavailability of some nodes, it still can be better than failing the query completely and returning nothing.
* **Yield**
Specifies the number of requests that were completed successfully, compared to the total number of attempted requests. Yield is different from the uptime, since, for example, a busy node is not down, but still can fail to respond to some of the requests